# 少量样本示例的提示词模板
## 1. 使用说明
在构建prompt时，可以通过构建一个少量示例列表去进一步格式化prompt，这是一种简单但强大的指导生成的方式，在某些情况下可以显著提高模型性能。
少量示例提示模板可以由一组示例或一个负责从定义的集合中选择一部分示例的示例选择器构建。
- 前者：使用FewShotPromptTemplate 或 FewShotChatMessagePromptTemplate
- 后者：使用 Example selectors（示例选择器）

举例1：未提供示例的情况

In [2]:
from langchain_openai import ChatOpenAI
import os
import dotenv

dotenv.load_dotenv()    # 加载当前目录下的 .env 文件
os.environ.setdefault("OPENAI_API_KEY", os.getenv("OPENAI_API_KEY"))
os.environ.setdefault("OPENAI_BASE_URL", os.getenv("OPENAI_BASE_URL"))

# 获取对话模型
model = ChatOpenAI(
    model="deepseek-chat",
    temperature=0.7,
    max_tokens=20
)

result = model.invoke("2 👇 9是多少？")
print(result)

content='你写的 “2 👇 9” 可能表示 **2 的 9 次方**，因为“👇”有时用来表示“向下”或“次方”的意思（类似上标）。  \n\n如果是这样：  \n\\[\n2^9 = 2 \\times 2 \\times 2 \\times 2 \\times 2 \\times 2 \\times 2 \\times 2 \\times 2 = 512\n\\]\n\n**答案：512**' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 99, 'prompt_tokens': 11, 'total_tokens': 110, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 11}, 'model_provider': 'openai', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': '84e2d4cd-ca63-4a4a-937a-51398f752813', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019c1d61-9280-7153-973a-774f62e52a61-0' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 11, 'output_tokens': 99, 'total_tokens': 110, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}}


举例2：使用FewShotPromptTemplate

In [5]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

# 创建一个 PromptTemplate 的实例
example_prompt = PromptTemplate.from_template(
    template="input:{input}\noutput:{output}",
)

# 提供示例
example = [
    {"input": "北京天气怎么样", "output": "北京市"},
    {"input": "南京下雨吗", "output": "南京市"},
    {"input": "西安热吗", "output": "西安市"}
]

# 创建一个 FewShotPromptTemplate 的实例
few_shot_template = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=example,
    suffix="input:{input}\noutput:",  # 声明在示例后面的提示词模板
    input_variables=["input"],
)

# 模型调用
from langchain_openai import ChatOpenAI
import os
import dotenv

dotenv.load_dotenv()    # 加载当前目录下的 .env 文件
os.environ.setdefault("OPENAI_API_KEY", os.getenv("OPENAI_API_KEY"))
os.environ.setdefault("OPENAI_BASE_URL", os.getenv("OPENAI_BASE_URL"))

# 获取对话模型
model = ChatOpenAI(
    model="deepseek-chat",
    temperature=0.7,
    max_tokens=20
)

result = model.invoke(few_shot_template.invoke({"input":"天津会下雨吗？"}))
print(result)

content='天津市' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 43, 'total_tokens': 44, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 43}, 'model_provider': 'openai', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': 'b3b5f7c4-9727-4f54-af54-3c707faf2e09', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019c1d6b-05f1-7f13-bcfe-52800d564071-0' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 43, 'output_tokens': 1, 'total_tokens': 44, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}}


举例3：

In [1]:
#1、创建提示模板
from langchain_core.prompts import PromptTemplate

# 创建提示模板，配置一个提示模板，将一个示例格式化为字符串
prompt_template = "你是一个数学专家,算式： {input} 值： {output} 使用： {description} "
# 这是一个提示模板，用于设置每个示例的格式
prompt_sample = PromptTemplate.from_template(prompt_template)
#2、提供示例
examples = [
    {"input": "2+2", "output": "4", "description": "加法运算"},
    {"input": "5-2", "output": "3", "description": "减法运算"},
]
#3、创建一个FewShotPromptTemplate对象
from langchain_core.prompts.few_shot import FewShotPromptTemplate

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=prompt_sample,
    suffix="你是一个数学专家,算式: {input} 值: {output}",
    input_variables=["input", "output"]
)
print(prompt.invoke({"input": "2*5", "output": "10"}))
#4、初始化大模型，然后调用
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()
os.environ.setdefault("OPENAI_API_KEY", os.getenv("OPENAI_API_KEY"))
os.environ.setdefault("OPENAI_BASE_URL", os.getenv("OPENAI_BASE_URL"))
chat_model = ChatOpenAI(
    model="deepseek-chat",
    temperature=0.7,
    max_tokens=20
)
result = chat_model.invoke(prompt.invoke({"input": "2*5", "output": "10"}))
print(result.content)  # 使用: 乘法运算

E:\shangdj\python\env\conda\envs\myPython3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


text='你是一个数学专家,算式： 2+2 值： 4 使用： 加法运算 \n\n你是一个数学专家,算式： 5-2 值： 3 使用： 减法运算 \n\n你是一个数学专家,算式: 2*5 值: 10'
你是一个数学专家，算式： 2×5 值： 10 使用： 乘法运算


# FewShotChatMessagePromptTemplate 的使用
除了 FewShotPromptTemplate 之外，FewShotChatMessagePromptTemplate 是专门为聊天对话场景设计的少样本（few-shot）提示模板，它继承自 FewShotPromptTemplate，但针对聊天消息的格式进行了优化